In [1]:
## import libraries
import os
from os import listdir
from os.path import isfile, join

import pandas as pd
from itertools import islice
import numpy as np

from skimage.external import tifffile
from skimage.external.tifffile import imread

import matplotlib.pyplot as plt

import ipywidgets as widgets

import random


from scipy.ndimage.morphology import binary_erosion as br
from skimage import morphology as skmor

from scipy import ndimage
from PIL import Image, ImageDraw

import numpy.ma as ma

from skimage.measure import label, regionprops

In [2]:
bigDir=r'Z:\CookLab\Liu\20190816_organizedData_MCM_loading'

In [13]:
newfile=f'{bigDir}\\new dataset\cellinfo_signals_v3.csv'
newdata=pd.read_csv(newfile)

In [4]:
orgfile=f'{bigDir}\\csv_signals_cell_data\\signal_data_total_v3.csv'
orgdata=pd.read_csv(orgfile)

# testing the volume of the masks

In [11]:
def calculate_volume (data,volumes):
    volumes = pd.DataFrame()
    i=0
    progBar=widgets.IntProgress(
        value=i,
        min=0,
        max=len(data),
        step=1,
        description='Progress:',
        orientation='horizontal'
    )
    display(progBar)

    for i,myCell in islice(data.iterrows(), i,None):
        path=myCell.path
        
        # nucleus - inner
        ## reading mask files
        innerNucleusPath = path.replace('data_tiff','erosion_nuc_mask')
        innerNucleusPath= innerNucleusPath.replace('.tif','_eroded_9^5_inner_mask_20.tif')
        try:
            innerNuc=imread(innerNucleusPath)    
        except:
            innerNucleusPath=innerNucleusPath.replace('.tif','.tiff')
            innerNuc=imread(innerNucleusPath)      
        ## making this image binary
        innerNuc=innerNuc.astype(bool)
        ## calculating the volume of the masks and entering into the dataframe
        volumes.at[i,"innerNuc"]=np.sum(innerNuc)
        
        # nucleus - outer
        ## reading mask files
        outerNucleusPath = path.replace('data_tiff','erosion_nuc_mask')
        outerNucleusPath=outerNucleusPath.replace('.tif','_eroded_9^5_outer_mask_20.tif')
        try:
            outerNuc=imread(outerNucleusPath)    
        except:
            outerNucleusPath=outerNucleusPath.replace('.tif','.tiff')
            outerNuc=imread(outerNucleusPath)
        ## making this image binary
        outerNuc=outerNuc.astype(bool)
        ## calculating the volume of the masks and entering into the dataframe. 
        volumes.at[i,"outerNuc"]=np.sum(outerNuc)
        
        # nucleus - total
        ## reading mask files
        totalNucleusPath = path.replace('data_tiff','segmentation_nucleus_Otsu')
        totalNucleusPath=totalNucleusPath.replace('.tif','_nucleus.tif')
        try:
            totalNuc=imread(totalNucleusPath)    
        except:
            totalNucleusPath=totalNucleusPath.replace('.tif','.tiff')
            totalNuc=imread(totalNucleusPath)
        ## masking this image binary
        totalNuc=totalNuc.astype(bool)
        ## calculating the volume of the masks and entering into the dataframe. 
        volumes.at[i,"totalNucleus"]=np.sum(totalNuc)
        
        # het - inner
        ## reading mask files
        innerHetPath = path.replace('data_tiff','segmentation_20_erosion')
        innerHetPath=innerHetPath.replace('.tif','_hetChrom_inner.tif')
        try:
            innerHet=imread(innerHetPath)    
        except:
            innerHetPath=innerHetPath.replace('.tif','.tiff')
            innerHet=imread(innerHetPath)
        ## masking this image binary
        innerHet=innerHet.astype(bool)
        ## calculating the volume of the masks and entering into the dataframe. 
        volumes.at[i,"innerHet"]=np.sum(innerHet)
        
        # het - outer
        ## reading mask files
        outerHetPath = path.replace('data_tiff','segmentation_20_erosion')
        outerHetPath=outerHetPath.replace('.tif','_hetChrom_outer.tif')
        try:
            outerHet=imread(outerHetPath)    
        except:
            outerHetPath=outerHetPath.replace('.tif','.tiff')
            outerHet=imread(outerHetPath)
        ## masking this image binary
        outerHet=outerHet.astype(bool)
        ## calculating the volume of the masks and entering into the dataframe.
        volumes.at[i,"outerHet"]=np.sum(outerHet)
        
        # het - total
        ## reading mask files
        totalHetPath = path.replace('data_tiff','segmentation_20')
        totalHetPath=totalHetPath.replace('.tif','_hetChrom.tif')
        try:
            totalHet=imread(totalHetPath)  
        except:
            totalHetPath=totalHetPath.replace('.tif','.tiff')
            totalHet=imread(totalHetPath)
        ## masking this image binary
        totalHet=totalHet.astype(bool)
        ## calculating the volume of the masks and entering into the dataframe.
        volumes.at[i,"totalHet"]=np.sum(totalHet)

        # eu - inner
        ## reading mask files
        innerEuPath = path.replace('data_tiff','segmentation_20_eu_erosion')
        innerEuPath=innerEuPath.replace('.tif','_euChrom_inner.tif')
        try:
            innerEu=imread(innerEuPath)
        except:
            innerEuPath=innerEuPath.replace('.tif','.tiff')
            innerEu=imread(innerEuPath)
        ## masking this image binary
        innerEu=innerEu.astype(bool)
        ## calculating the volume of the masks and entering into the dataframe
        volumes.at[i,"innerEu"]=np.sum(innerEu)
        
        # eu - outer
        ## reading mask files
        outerEuPath =path.replace('data_tiff','segmentation_20_eu_erosion')
        outerEuPath=outerEuPath.replace('.tif','_euChrom_outer.tif')
        try:
            outerEu=imread(outerEuPath)
        except:
            outerEuPath=outerEuPath.replace('.tif','.tiff')
            outerEu=imread(outerEuPath)
        ## masking this image binary
        outerEu=outerEu.astype(bool)
        ## calculating the volume of the masks and entering into the dataframe
        volumes.at[i,"outerEu"]=np.sum(outerEu)
        
        # eu - total
        ## reading mask files
        totalEuPath = path.replace('data_tiff','segmentation_20_eu')
        totalEuPath=totalEuPath.replace('.tif','_euChrom.tif')
        try:
            totalEu=imread(totalEuPath)
        except:
            totalEuPath=totalEuPath.replace('.tif','.tiff')
            totalEu=imread(totalEuPath)
        ## masking this image binary
        totalEu=totalEu.astype(bool)
        ## calculating the volume of the masks and entering into the dataframe
        volumes.at[i,"totalEu"]=np.sum(totalEu)
        
        progBar.value=progBar.value+1
        
    return volumes
        

In [23]:
def testRatio (volDf, part1, part2, total):
    return np.sum(volDf[part1] + volDf[part2]) / np.sum(volDf[total]) == 1.0
        

In [14]:
newdata_volumes=pd.DataFrame()
newdata_vol=calculate_volume(newdata,newdata_volumes)

IntProgress(value=0, description='Progress:', max=324)

In [16]:
newdata_vol.head()

,innerNuc,outerNuc,totalNucleus,innerHet,outerHet,totalHet,innerEu,outerEu,totalEu
0,47037.0,216157.0,263194.0,29163.0,23479.0,52642.0,17874.0,192678.0,210552.0
1,76346.0,259643.0,335989.0,39663.0,27545.0,67208.0,36683.0,232098.0,268781.0
2,74826.0,255724.0,330550.0,39304.0,26820.0,66124.0,35522.0,228904.0,264426.0
3,28597.0,195272.0,223869.0,19580.0,25209.0,44789.0,9017.0,170063.0,179080.0
4,57497.0,259717.0,317214.0,36637.0,26823.0,63460.0,20860.0,232894.0,253754.0


In [27]:
orgdata_volumes=pd.DataFrame()
orgdata_vol=calculate_volume(orgdata,orgdata_volumes)

IntProgress(value=0, description='Progress:', max=300)

In [34]:
orgdata_vol.head()

,innerNuc,outerNuc,totalNucleus,innerHet,outerHet,totalHet,innerEu,outerEu,totalEu
0,121203.0,338695.0,459898.0,56704.0,35278.0,91982.0,64499.0,303417.0,367916.0
1,113058.0,323479.0,436537.0,53828.0,33491.0,87319.0,59230.0,289988.0,349218.0
2,110924.0,286116.0,397040.0,54924.0,24487.0,79411.0,56000.0,261629.0,317629.0
3,112322.0,309306.0,421628.0,53664.0,30679.0,84343.0,58658.0,278627.0,337285.0
4,283573.0,530699.0,814272.0,112732.0,50131.0,162863.0,170841.0,480568.0,651409.0


# testRatio(volumedataframe, part1, part2, total)
This function takes in the dataframe that has volumes of the masks, string value of the compartment 1, compartment 2 and the total which the ratio of the sum volume of compartment 1 and 2 and the total volume should be 1. IF the ratio is 1, the function returns **True**, else returns **False**. 

## new data tests

In [28]:
testRatio(newdata_vol,"innerNuc","outerNuc","totalNucleus")

True

In [29]:
testRatio(newdata_vol,"innerHet","outerHet","totalHet")

False

In [33]:
np.sum(newdata_vol["innerHet"]+newdata_vol["outerHet"]) / np.sum(newdata_vol["totalHet"])

0.9980484018970869

In [30]:
testRatio(newdata_vol,"innerEu","outerEu","totalEu")

True

In [31]:
testRatio(newdata_vol,"innerHet","innerEu","innerNuc")

True

In [32]:
testRatio(newdata_vol,"outerHet","outerEu","outerNuc")

True

## original data tests

In [35]:
testRatio(orgdata_vol,"innerNuc","outerNuc","totalNucleus")

True

In [36]:
testRatio(orgdata_vol,"innerHet","outerHet","totalHet")

True

In [38]:
testRatio(orgdata_vol,"innerEu","outerEu","totalEu")

True

In [39]:
testRatio(orgdata_vol,"innerHet","innerEu","innerNuc")

True

In [40]:
testRatio(orgdata_vol,"outerHet","outerEu","outerNuc")

True